<a href="https://colab.research.google.com/github/AliAli7299/PowerBI-project/blob/main/SQL-Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**In this section, we will be verifying the values that were deducted from PowerBI using SQL. We will be checking total revenue with respect to products, product categories, subcategories, % profit and change in revenue over the years.**

In [60]:
sales <- read.csv("/content/sales.csv")
product <- read.csv("/content/Product.csv")
productcategory <- read.csv("/content/ProductCategory.csv")
productsubcategory <- read.csv("/content/ProductSubcategory.csv")
stores <- read.csv("/content/Stores.csv")
countries <- read.csv("/content/Geography.csv")

In [3]:
install.packages("DBI")
install.packages("RSQLite")
library(DBI)
library(RSQLite)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’




In [62]:
con <- dbConnect(SQLite(), "my_database.sqlite")

In [6]:
dbWriteTable(con, "sales", sales, overwrite = TRUE)
dbWriteTable(con, "productcategory", productcategory, overwrite = TRUE)
dbWriteTable(con, "productsubcategory", productsubcategory, overwrite = TRUE)
dbWriteTable(con, "stores", stores, overwrite = TRUE)
dbWriteTable(con, "countries", countries, overwrite = TRUE)
dbWriteTable(con,"product",product,overwrite = TRUE)

# Let's check if the sum of revenue matches our dashboard


In [8]:
query1 <- "SELECT SUM(SalesAmount) FROM sales"
result1 <- dbGetQuery(con, query1)

value <- as.numeric(result1[[1]])

formatted_result <- format(value, big.mark = ",", nsmall = 2, scientific = FALSE)

cat("Total Revenue Amount: ", formatted_result, "\n")

Total Revenue Amount:  8,341,234,294.00 


**The sum of revenue matches the numbers on our dashboard, where the total revenue was $ 8.34 B.**
**Next, we will check The sum of revenue with respect to each category.**

In [9]:
query2 <- "SELECT SUM(SalesAmount) as SumofSales,productCategory from sales
join product on product.productKey = sales.productKey
join productSubCategory on product.productSubcategoryKey = ProductSubcategory.productSubcategoryKey
join productCategory on productCategory.productCategoryKey = ProductSubcategory.productcategoryKey
Group by ProductCategory
ORDER BY SumofSales DESC
"

result2 <- dbGetQuery(con,query2)
result2$SumofSales <- formatC(as.numeric(result2$SumofSales), format = "f", digits = 0, big.mark = ",")
result2

SumofSales,ProductCategory
<chr>,<chr>
"3,209,419,288",Computers
"2,562,027,122",Cameras and camcorders
"1,360,118,988",TV and Video
"892,237,275",Cell phones
"165,812,823","Music, Movies and Audio Books"
"151,618,798",Audio


# **This data frame shows the total sum of sales for six product categories, with Computers having the highest and Audio the lowest.This validates the numbers deduced from PowerBI**

#**Let's try to validate the sum of sales with respect to each country, and whether it matches our dashboard.**

In [10]:
query3 <- "SELECT SUM(SalesAmount) As sumofsales,RegionCountryname from  sales
join stores on sales.StoreKey = stores.storekey
join countries on countries.Geographykey = stores.geographykey
group by regioncountryname
order by sumofsales DESC LIMIT 10"


result3 <- dbGetQuery(con,query3)
result3$sumofsales <- formatC(as.numeric(result3$sumofsales), format = "f", digits = 0, big.mark = ",")
result3

sumofsales,RegionCountryName
<chr>,<chr>
"4,756,355,186",United States
"1,063,858,277",China
"662,755,945",Germany
"433,928,946",France
"220,900,959",United Kingdom
"175,942,974",Canada
"163,474,137",Japan
"79,166,726",Australia
"77,874,035",India


# **Let's do the same for product subcategory.**

In [13]:
query4 <- "SELECT SUM(SalesAmount) as SumofSales,productSubcategory from sales
join product on product.productKey = sales.productKey
join productSubCategory on product.productSubcategoryKey = ProductSubcategory.productSubcategoryKey
Group by ProductSubcategory
ORDER BY SumofSales DESC LIMIT 10
"

result4 <- dbGetQuery(con,query4)
result4$SumofSales <- formatC(as.numeric(result4$SumofSales), format = "f", digits = 0, big.mark = ",")
result4

SumofSales,ProductSubcategory
<chr>,<chr>
"1,335,302,129",Camcorders
"1,107,194,324",Projectors & Screens
"933,128,637",Laptops
"809,994,029",Digital SLR Cameras
"709,116,951",Home Theater System
"508,192,585",Desktops
"423,388,671",Smart phones & PDAs
"365,082,773",Digital Cameras
"307,378,938",Televisions


# In the next step, we'll calculate the % profit for product subcategories.

In [19]:
query5 <- " SELECT  SUM(SalesAmount - TotalCost) / SUM(SalesAmount) As percent_profit, productSubcategory from sales
join product on product.productKey = sales.productKey
join productSubCategory on product.productSubcategoryKey = ProductSubcategory.productSubcategoryKey
Group by ProductSubcategory
ORDER BY percent_profit ASC LIMIT 10"

result5 <- dbGetQuery(con,query5)
result5$percent_profit <- paste0(formatC(as.numeric(result5$percent_profit) * 100, format = "f", digits = 2), "%")

result5

percent_profit,ProductSubcategory
<chr>,<chr>
51.38%,Desktops
52.18%,Cameras & Camcorders Accessories
52.44%,Computers Accessories
52.52%,Cell phones Accessories
52.75%,Car Video
53.34%,VCD & DVD
54.05%,Recording Pen
54.18%,Home Theater System
55.37%,Touch Screen Phones


# **Finally, we need to calculate change in % revenue between 2011 and 2013.**

In [63]:
query6 <- "
WITH sales_2011 AS (
  SELECT productKey, SUM(SalesAmount) AS revenue_2011
  FROM sales
  WHERE year = 2011
  GROUP BY productKey
),
sales_2013 AS (
  SELECT productKey, SUM(SalesAmount) AS revenue_2013
  FROM sales
  WHERE year = 2013
  GROUP BY productKey
)
SELECT
  p.productName,
  ROUND(
    ((s13.revenue_2013 * 100.0 / NULLIF(s11.revenue_2011, 0)) - 100),
    2
  ) AS percent_change
FROM sales_2011 s11
JOIN sales_2013 s13 ON s11.productKey = s13.productKey
JOIN product p ON p.productKey = s11.productKey
ORDER BY percent_change ASC
LIMIT 10
"

result6 <- dbGetQuery(con, query6)

result6$percent_change <- paste0(
  formatC(as.numeric(result6$percent_change), format = 'f', digits = 2),
  "%"
)

result6


ProductName,percent_change
<chr>,<chr>
Adventure Works Desktop PC1.80 ED182 Brown,-88.70%
Adventure Works Desktop PC1.80 ED180 Black,-88.66%
Adventure Works Desktop PC1.80 ED180 Brown,-88.49%
Adventure Works Desktop PC1.80 ED182 Silver,-88.38%
Adventure Works Desktop PC1.80 ED180 White,-88.29%
Adventure Works Desktop PC3.0 MS300 Silver,-87.70%
WWI Desktop PC3.0 M0300 Silver,-87.56%
Adventure Works Desktop PC1.80 ED182 White,-87.38%
Adventure Works Desktop PC1.80 ED182 Black,-86.85%


In [64]:
dbDisconnect(con)

# **Sum of sales in product subcategory, % profit, and change in revenue all match the numbers in PowerBI. this way we can make sure that PowerBI gave us the correct output.**